# Polars - Fast Data Analysis in Python: Part 1: DataFrames

Polars is a highly performant library for manipulating structured data. Polars is one of the best performing solutions available when it comes to clearning, preparing, and transforming medium-sized datasets (>10GB), which is typically where Pandas begins to bog down.

The objective of this tutorial is to cover the most important features of Polars.

This is Part 1 of this tutorial which introduces Polars' most important data structure, the **DataFrame**, and explains how to import data, query, and clean data.

Let us begin by importing `polars` and `pandas` into our namespace.

In [1]:
import polars as pl
import pandas as pd

## The DataFrame

The DataFrame is a table with an index that labels rows and columns. A DataFrame typically stores structured data, such as numeric values or text labels.

### Elements of a DataFrame

![Drawing of a DataFrame](https://lscm.s3.amazonaws.com/pandas/dataframe_in_a_nutshell.png)

#### Data Types
A DataFrame organizes data in rows and columns, whereby each data point occupies a separate row. Rows are expected to have the same length, and columns are expected to be of the same type.

Columns can hold numeric data, like real values (`f64`) or integers (`i64`), as well as categorical data, like text (`type=str`). They may even contain objects, for example `dt` objects to store date and time.

#### Rows and Columns
Each row and each column has a unique label which gives us a row index (the `Polars.row`) and a column index (`Polars.col`). We can uses these indices to select individual values from the DataFrame. Typically, row index labels are integers (or time stamps) and column index labels are names.

Internally, data is stored in Apache Arrow, a columnar data format that facilitates in-memory exchange of data between system processes.

#### Missing Values
If a data point does not show a value for a specific column, we refer to this as a missing value. Missing values are labeled as `null` which stands for not a number.




### Creating a DataFrame

We begin with a simple DataFrame that stores information of the continents on Earth, like their population and their land area.

Suppose our data is organized in a dictionary, with column labels as keys and values as lists that contain, continent names, population, and area.

In [2]:
mydata = dict(
    continent=['Africa', 'Antartica', 'Asia', 'Australia', 'Europe','North America', 'South America'],
    population=[1216000000.0, None, 4463000000.0, 24130000.0, 741400000.0, 579000000.0, 422500000.0],
    area=[30221532, 14000000, 44579000, 8525989, 10180000, 24709000, 17840000]
    )

Let us create a DataFrame with the continent data as input.

In [3]:
df = pl.DataFrame(data=mydata)
df

continent,population,area
str,f64,i64
"""Africa""",1.2160e9,30221532
"""Antartica""",null,14000000
"""Asia""",4.4630e9,44579000
"""Australia""",2.413e7,8525989
"""Europe""",7.414e8,10180000
"""North America""",5.79e8,24709000
"""South America""",4.225e8,17840000


As we can see, the DataFrame shows the column name in the header and the values in the columns. Additionally, it shows the data type for each columns.

### Indexing

We can now access the data in all kinds of ways. For example, by getting a single column.

In [4]:
df.select("population")

population
f64
1.2160e9
null
4.4630e9
2.413e7
7.414e8
5.79e8
4.225e8


We can also select multiple columns, which we can do by passing a list of column names.

In [5]:
df.select("population","area")

population,area
f64,i64
1.2160e9,30221532
null,14000000
4.4630e9,44579000
2.413e7,8525989
7.414e8,10180000
5.79e8,24709000
4.225e8,17840000


We can also use the function `polars.col` to create an object.

In [6]:
df.select(pl.col("population"))

population
f64
1.2160e9
null
4.4630e9
2.413e7
7.414e8
5.79e8
4.225e8


To select rows, where the column attributes take on specified values, we use the function `filter`. To create a boolean expression, we must use the column object from the previous step.

In [7]:
df.filter(pl.col("continent")=="Europe")#.select("population")

continent,population,area
str,f64,i64
"""Europe""",7.414e8,10180000


We can even combine the two functions in a chain. Better use line breaks here and start a new row with each function call.

In [8]:
df.filter(
    pl.col("continent")=="Europe"
 ).select(
  "population"
 )

population
f64
7.414e8


We can also select a row by using its index, which returns the content of the row as a tuple.

In [9]:
df.row(0)

('Africa', 1216000000.0, 30221532)

Or slice the DataFrame by choosing only a range of rows.

In [10]:
df.slice(1,3)

continent,population,area
str,f64,i64
"""Antartica""",null,14000000
"""Asia""",4.4630e9,44579000
"""Australia""",2.413e7,8525989


We can even combine select and filter in more creative ways.

In [11]:
df.select(
    "continent","population"
    ).filter(
        pl.col("population")>1000000000
    )

continent,population
str,f64
"""Africa""",1.2160e9
"""Asia""",4.4630e9


We can also create more complex boolean slices by logically combining boolean expressions:

- logical AND `&`
- logical OR `|`
- logical NOT `~`

Each logical expression must be sourrounded by parantheses, for example, `(...) & (...)`.

In [12]:
df.select(
    "continent","population"
    ).filter(
        (pl.col("population")>100000000) & (pl.col("population") < 1000000000)
    )

continent,population
str,f64
"""Europe""",7.414e8
"""North America""",5.79e8
"""South America""",4.225e8


Unfortunately, complex selections become quite hard to

---

read in this way. There are helper functions, such as `is_in` or the column attribute `str`.

In [13]:
df.select(
    "continent","population"
).filter(
    pl.col("continent").is_in(["Europe","Asia"])
)

continent,population
str,f64
"""Asia""",4.4630e9
"""Europe""",7.414e8


In [14]:
df.select(
    "continent","population"
    ).filter(
        pl.col("continent").str.starts_with("A")
    )

continent,population
str,f64
"""Africa""",1.2160e9
"""Antartica""",null
"""Asia""",4.4630e9
"""Australia""",2.413e7


Suppose we now want to include the names of the most populous cities. Using the same order as the continents, we store these in an additional list.

In [15]:
cities = ["Lagos", "McMurdo Station", "Shanghai", "Sydney", "Moscow", "Mexico City", "São Paulo"]
cities

['Lagos',
 'McMurdo Station',
 'Shanghai',
 'Sydney',
 'Moscow',
 'Mexico City',
 'São Paulo']

Instead of creating a new DataFrame, we can simply append this data to our DataFrame by adding it as an additional column.

In [16]:
df = df.with_columns(most_populous_city=pl.lit(cities))
df

/tmp/ipykernel_2813/2938771235.py:1: DeprecationWarning: Behavior for `lit` will change for sequence inputs. The result will change to be a literal of type List. To retain the old behavior, pass a Series instead, e.g. `Series(sequence)`.
  df = df.with_columns(most_populous_city=pl.lit(cities))


continent,population,area,most_populous_city
str,f64,i64,str
"""Africa""",1.2160e9,30221532,"""Lagos"""
"""Antartica""",null,14000000,"""McMurdo Statio…"
"""Asia""",4.4630e9,44579000,"""Shanghai"""
"""Australia""",2.413e7,8525989,"""Sydney"""
"""Europe""",7.414e8,10180000,"""Moscow"""
"""North America""",5.79e8,24709000,"""Mexico City"""
"""South America""",4.225e8,17840000,"""São Paulo"""


#### Exercises

1. Get the name of the most populous city of the continent with an area of less than 10,000,000.

In [17]:
df.filter(pl.col("area")<10000000).select("most_populous_city")

most_populous_city
str
"""Sydney"""


2. Find the row that contains NaNs in the column population using the boolean function `isna` and display the name of the continent. (Remark: `isna` and `isnull` behave identically.)




In [18]:
df.filter(pl.any(pl.all().is_null()))

/tmp/ipykernel_2813/1227619148.py:1: DeprecationWarning: using `any` for horizontal computation is deprecated. Use `any_horizontal` instead.
  df.filter(pl.any(pl.all().is_null()))


continent,population,area,most_populous_city
str,f64,i64,str
"""Antartica""",null,14000000,"""McMurdo Statio…"


3. Return the name of those continents whose largest city starts with an "M". Lagos is the most populous city of Africa.

In [19]:
df.filter(pl.col("most_populous_city").str.starts_with("M"))

continent,population,area,most_populous_city
str,f64,i64,str
"""Antartica""",null,14000000,"""McMurdo Statio…"
"""Europe""",7.414e8,10180000,"""Moscow"""
"""North America""",5.79e8,24709000,"""Mexico City"""


## Importing Data

In most cases, you will not have data lying around in Python dictionaries, but you will want to import them from some other data source, like a CSV file, an Excel workbook, a web page, a database, etc.

To load data from an external data source into a DataFrame, Polars offers a series of methods, all starting with *read*:

- `read_excel`: Import from Excel file
- `read_csv`: Import from CSV (comma-separated values) file
-`read_json`: Import from JSON-formated file
-`read_sql`: Import from a database
- ...

Let us stick with our continent example and get some more data from Wikipedia: https://en.wikipedia.org/wiki/Continent.



### CSV Import

To skip the caveats of web scraping for the moment, we will import the continent data as CSV from a URL which contains a clean data set.

The method `read_csv` parses a text file containing comma-separated  values into a DataFrame. Of course, there is no standard way to store a CSV. Some text files use commas others use tabs to separate values; some come with an index column other do not; some CSVs do not even have a header row. Polars will make some default assumptions about how the data is formated, but often we must tweak these arguments until the result looks right.

#### Exercises

1. Use `pl.read_csv` to import the CSV that is located at this URL: https://lscm.s3.amazonaws.com/pandas/wikipedia_continents_data.csv.


In [20]:
pl.read_csv("https://lscm.s3.amazonaws.com/pandas/wikipedia_continents_data.csv")

Continent Area (km2) Area (sq mi) Pct of total area Population Pct of total population Most populous city
str
"""0 Africa 30370…"
"""1 Antarctica 1…"
"""2 Asia 4457900…"
"""3 Europe 10180…"
"""4 North Americ…"
"""5 South Americ…"
"""6 Australia 86…"


2. The CSV is separated by tabs (indicated by regular expression `\t`) and it contains a header row as well as an index column. Use the arguments and `separator` and `columns `to get a properly formatted DataFrame.



In [21]:
pl.read_csv(source="https://lscm.s3.amazonaws.com/pandas/wikipedia_continents_data.csv", separator="\t")

,Continent,Area (km2),Area (sq mi),Pct of total area,Population,Pct of total population,Most populous city
i64,str,i64,i64,f64,f64,f64,str
0,"""Africa""",30370000,11730000,20.4,1.2879e9,16.9,"""Lagos"""
1,"""Antarctica""",14000000,5400000,9.2,null,0.0,"""McMurdo Statio…"
2,"""Asia""",44579000,17212000,29.5,4.5451e9,59.5,"""Shanghai"""
3,"""Europe""",10180000,3930000,6.8,7.42648e8,9.7,"""Moscow"""
4,"""North America""",24709000,9540000,16.5,5.87615e8,7.7,"""Mexico City"""
5,"""South America""",17840000,6890000,12.0,4.2824e8,5.6,"""São Paulo"""
6,"""Australia""",8600000,3300000,5.9,4.1261e7,0.5,"""Sydney"""


3. Only read columns that contain the continent name and its population. Take a look at the [documentation](https://pola-rs.github.io/polars/py-polars/html/reference/api/polars.read_csv.html) to see which options are available.

In [22]:
pl.read_csv("https://lscm.s3.amazonaws.com/pandas/wikipedia_continents_data.csv",
            separator="\t", columns=["Continent", "Area (km2)"])

Continent,Area (km2)
str,i64
"""Africa""",30370000
"""Antarctica""",14000000
"""Asia""",44579000
"""Europe""",10180000
"""North America""",24709000
"""South America""",17840000
"""Australia""",8600000


### Import from Excel

Unless, data has been explicitly exported, manual data entry and manipulation is typically done in spreadsheets, for example, Microsoft Excel.

Let us now import the same data from an Excel file from the same URL. The table with the population data is located in the worksheet *Population*. We therefore use the argument `sheet_name="Population"` to select the correct sheet.

In [23]:
df = pl.from_pandas(pd.read_excel("https://lscm.s3.amazonaws.com/pandas/wikipedia_continents_data.xlsx", sheet_name="Population"))
df

Continent,Population,Pct of total population
str,f64,f64
"""Africa""",1.2879e9,16.9
"""Antarctica""",null,0.0
"""Asia""",4.5451e9,59.5
"""Europe""",7.42648e8,9.7
"""North America""",5.87615e8,7.7
"""South America""",4.2824e8,5.6
"""Australia""",4.1261e7,0.5


### Webpage Scraping

To get the lates figures of population data available on Wikipedia, we can also import the continent data directly.

When importing data from web pages, you can use quite advanced libraries, such as [Requests](https://requests.readthedocs.io/en/master/) or [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) (it is named like that!), but for simple pages that contain data in regular HTML tables, this is often not needed. Pandas `read_html` can read an HTML webpage and scan the page for tables. What we get in return are all tables on that page parsed into a list of DataFrames.

There are multiple tables that can be found under https://en.wikipedia.org/wiki/Continent. To ensure that this notebooks still works even if the page changes, we'll use a fixed snapshot. We are only interested in the one that summarizes area and population, which is the one with the index 2.

In [24]:
df = pl.from_pandas(pd.read_html(io="https://en.wikipedia.org/w/index.php?title=Continent&oldid=1003389784")[2])
df

Continent,Unnamed: 1,Area (km2) [failed verification],Area (sq mi) [failed verification],Percent total landmass [failed verification],Population,Percent total pop.,Most populous city (proper)
str,str,i64,i64,str,str,str,str
"""Africa""","""[note 1]""",30370000,11730000,"""20.4%""","""1287920000""","""16.9%""","""Lagos, Nigeria…"
"""Antarctica""","""[note 2]""",14000000,5400000,"""9.2%""","""4,490[42]""","""0.0%""","""McMurdo Statio…"
"""Asia""","""[note 3]""",44579000,17212000,"""29.5%""","""4545133000""","""59.5%""","""Shanghai, Chin…"
"""Europe""","""[note 4]""",10180000,3930000,"""6.8%""","""742648000""","""9.7%""","""Moscow, Russia…"
"""North America""","""[note 5]""",24709000,9540000,"""16.5%""","""587615000""","""7.7%""","""Mexico City, M…"
"""South America""",null,17840000,6890000,"""12.0%""","""428240000""","""5.6%""","""São Paulo, Bra…"
"""Australia""","""[note 6]""",8600000,3300000,"""5.9%""","""41261000""","""0.5%""","""Sydney, Austra…"


## Data Cleaning

The DataFrame that was generated from importing the Wikipedia page still contains a number of errors:

- There is an unnamed column that has no data.
- A few values contain references to footnotes.
- The percentage values include the percentage sign which Python does not understand

As we cannot work with the data as-is, the data needs to be cleaned.

Quite some time in any data science project goes into cleaning data and making sure that it is free of errors.

**Important**: When manipulating a DataFrame, we must know that any operation always returns a copy of the DataFrame and does not change the original DataFrame. To store the new DataFrame, it is common practice to reassign the variable name the return DataFrame.

### Deleting Rows and Columns

There is one column called `Unnamed: 1` that does not seem to contain useful information. Let us  therfore drop this column.

In [25]:
df = df.drop("Unnamed: 1")
df

Continent,Area (km2) [failed verification],Area (sq mi) [failed verification],Percent total landmass [failed verification],Population,Percent total pop.,Most populous city (proper)
str,i64,i64,str,str,str,str
"""Africa""",30370000,11730000,"""20.4%""","""1287920000""","""16.9%""","""Lagos, Nigeria…"
"""Antarctica""",14000000,5400000,"""9.2%""","""4,490[42]""","""0.0%""","""McMurdo Statio…"
"""Asia""",44579000,17212000,"""29.5%""","""4545133000""","""59.5%""","""Shanghai, Chin…"
"""Europe""",10180000,3930000,"""6.8%""","""742648000""","""9.7%""","""Moscow, Russia…"
"""North America""",24709000,9540000,"""16.5%""","""587615000""","""7.7%""","""Mexico City, M…"
"""South America""",17840000,6890000,"""12.0%""","""428240000""","""5.6%""","""São Paulo, Bra…"
"""Australia""",8600000,3300000,"""5.9%""","""41261000""","""0.5%""","""Sydney, Austra…"


### Changing the Index

The column names are a bit unhandy. Let us replace them with more readable names. One way of doing this is by changing the column names directly.

In [26]:
df = df.rename({
    "Area (km2) [failed verification]":"Area (km2)",
    "Area (sq mi) [failed verification]":"Area (sq mi)",
    "Percent total landmass [failed verification]":"Pct area",
    "Percent total pop.": "Pct population",
    "Most populous city (proper)":"Most populous city"
    })
df

Continent,Area (km2),Area (sq mi),Pct area,Population,Pct population,Most populous city
str,i64,i64,str,str,str,str
"""Africa""",30370000,11730000,"""20.4%""","""1287920000""","""16.9%""","""Lagos, Nigeria…"
"""Antarctica""",14000000,5400000,"""9.2%""","""4,490[42]""","""0.0%""","""McMurdo Statio…"
"""Asia""",44579000,17212000,"""29.5%""","""4545133000""","""59.5%""","""Shanghai, Chin…"
"""Europe""",10180000,3930000,"""6.8%""","""742648000""","""9.7%""","""Moscow, Russia…"
"""North America""",24709000,9540000,"""16.5%""","""587615000""","""7.7%""","""Mexico City, M…"
"""South America""",17840000,6890000,"""12.0%""","""428240000""","""5.6%""","""São Paulo, Bra…"
"""Australia""",8600000,3300000,"""5.9%""","""41261000""","""0.5%""","""Sydney, Austra…"


### Converting Data Types

While some columns contain only numeric value, most of them do not, so that Polars will treat those columns as categorical data. We must therefore convert these values into floats or ints.

The most straightforward way is to convert categorical columns into numeric ones while ignoring values that cannot be parsed. This can be achieved with the function `cast`. By setting argument `strict="False"`, any value that cannot be converted to a number will be set to `null`.



In [27]:
df = df.with_columns(pl.col("Population").cast(pl.Int64, strict=False))
df

Continent,Area (km2),Area (sq mi),Pct area,Population,Pct population,Most populous city
str,i64,i64,str,i64,str,str
"""Africa""",30370000,11730000,"""20.4%""",1287920000,"""16.9%""","""Lagos, Nigeria…"
"""Antarctica""",14000000,5400000,"""9.2%""",null,"""0.0%""","""McMurdo Statio…"
"""Asia""",44579000,17212000,"""29.5%""",4545133000,"""59.5%""","""Shanghai, Chin…"
"""Europe""",10180000,3930000,"""6.8%""",742648000,"""9.7%""","""Moscow, Russia…"
"""North America""",24709000,9540000,"""16.5%""",587615000,"""7.7%""","""Mexico City, M…"
"""South America""",17840000,6890000,"""12.0%""",428240000,"""5.6%""","""São Paulo, Bra…"
"""Australia""",8600000,3300000,"""5.9%""",41261000,"""0.5%""","""Sydney, Austra…"



Since categorical columns are strings, we can use string manipulation techniques to turn them into a column of strings that can be converted to numeric values. This time we

In [28]:
df = df.with_columns(pl.col("Pct population").str.replace("%","").cast(pl.Float64, strict=False))
df = df.with_columns(pl.col("Pct area").str.replace("%","").cast(pl.Float64, strict=False))
df

Continent,Area (km2),Area (sq mi),Pct area,Population,Pct population,Most populous city
str,i64,i64,f64,i64,f64,str
"""Africa""",30370000,11730000,20.4,1287920000,16.9,"""Lagos, Nigeria…"
"""Antarctica""",14000000,5400000,9.2,null,0.0,"""McMurdo Statio…"
"""Asia""",44579000,17212000,29.5,4545133000,59.5,"""Shanghai, Chin…"
"""Europe""",10180000,3930000,6.8,742648000,9.7,"""Moscow, Russia…"
"""North America""",24709000,9540000,16.5,587615000,7.7,"""Mexico City, M…"
"""South America""",17840000,6890000,12.0,428240000,5.6,"""São Paulo, Bra…"
"""Australia""",8600000,3300000,5.9,41261000,0.5,"""Sydney, Austra…"


The column with the most populous city still contains a citation marker that could be removed. This calls for a more tailored approach. Let us define a little lambda expression that removes the brackets and anything inside. To do this, let us write a little function that splits a text string at every occurance of `[` and returns the first element of the resulting list of strings.


In [29]:
"Moscow, Russia[43]".split("[")[0]

'Moscow, Russia'

In [30]:
df = df.with_columns(pl.col("Most populous city").str.replace(r"\[.*\]",""))
df

Continent,Area (km2),Area (sq mi),Pct area,Population,Pct population,Most populous city
str,i64,i64,f64,i64,f64,str
"""Africa""",30370000,11730000,20.4,1287920000,16.9,"""Lagos, Nigeria…"
"""Antarctica""",14000000,5400000,9.2,null,0.0,"""McMurdo Statio…"
"""Asia""",44579000,17212000,29.5,4545133000,59.5,"""Shanghai, Chin…"
"""Europe""",10180000,3930000,6.8,742648000,9.7,"""Moscow, Russia…"
"""North America""",24709000,9540000,16.5,587615000,7.7,"""Mexico City, M…"
"""South America""",17840000,6890000,12.0,428240000,5.6,"""São Paulo, Bra…"
"""Australia""",8600000,3300000,5.9,41261000,0.5,"""Sydney, Austra…"


### Replacing Missing Values

Before we can begin with our data analyses, we must verify whether our DataFrame contains missing values which are indicated as `NaN`. There are three ways of dealing with missing values:

1. Dropping the rows (or columns) that contain the missing values.
2. Replacing the missing values with actual values.
3. Leaving the missing values but avoiding operations that do not accept `NaN`s.

#### Exercises

1. Drop all rows that contain `NaN` by calling the function `dropna`.

In [31]:
df.filter(~pl.any(pl.all().is_null()))

/tmp/ipykernel_2813/2926645026.py:1: DeprecationWarning: using `any` for horizontal computation is deprecated. Use `any_horizontal` instead.
  df.filter(~pl.any(pl.all().is_null()))


Continent,Area (km2),Area (sq mi),Pct area,Population,Pct population,Most populous city
str,i64,i64,f64,i64,f64,str
"""Africa""",30370000,11730000,20.4,1287920000,16.9,"""Lagos, Nigeria…"
"""Asia""",44579000,17212000,29.5,4545133000,59.5,"""Shanghai, Chin…"
"""Europe""",10180000,3930000,6.8,742648000,9.7,"""Moscow, Russia…"
"""North America""",24709000,9540000,16.5,587615000,7.7,"""Mexico City, M…"
"""South America""",17840000,6890000,12.0,428240000,5.6,"""São Paulo, Bra…"
"""Australia""",8600000,3300000,5.9,41261000,0.5,"""Sydney, Austra…"


Or, we use a more convenient function that achieves the same.

In [32]:
df = df.drop_nulls()
df

Continent,Area (km2),Area (sq mi),Pct area,Population,Pct population,Most populous city
str,i64,i64,f64,i64,f64,str
"""Africa""",30370000,11730000,20.4,1287920000,16.9,"""Lagos, Nigeria…"
"""Asia""",44579000,17212000,29.5,4545133000,59.5,"""Shanghai, Chin…"
"""Europe""",10180000,3930000,6.8,742648000,9.7,"""Moscow, Russia…"
"""North America""",24709000,9540000,16.5,587615000,7.7,"""Mexico City, M…"
"""South America""",17840000,6890000,12.0,428240000,5.6,"""São Paulo, Bra…"
"""Australia""",8600000,3300000,5.9,41261000,0.5,"""Sydney, Austra…"


2. Now drop the column with area in square miles and rename the column "Area (km2)" to "Area".

In [33]:
df = (df
      .drop("Area (sq mi)")
      .rename({"Area (km2)":"area"})
)
df

Continent,area,Pct area,Population,Pct population,Most populous city
str,i64,f64,i64,f64,str
"""Africa""",30370000,20.4,1287920000,16.9,"""Lagos, Nigeria…"
"""Asia""",44579000,29.5,4545133000,59.5,"""Shanghai, Chin…"
"""Europe""",10180000,6.8,742648000,9.7,"""Moscow, Russia…"
"""North America""",24709000,16.5,587615000,7.7,"""Mexico City, M…"
"""South America""",17840000,12.0,428240000,5.6,"""São Paulo, Bra…"
"""Australia""",8600000,5.9,41261000,0.5,"""Sydney, Austra…"


3. Strip the country information from the column  "Most populous city".

In [34]:
df.with_columns(pl.col("Most populous city").str.replace(r",(.*)",""))

Continent,area,Pct area,Population,Pct population,Most populous city
str,i64,f64,i64,f64,str
"""Africa""",30370000,20.4,1287920000,16.9,"""Lagos"""
"""Asia""",44579000,29.5,4545133000,59.5,"""Shanghai"""
"""Europe""",10180000,6.8,742648000,9.7,"""Moscow"""
"""North America""",24709000,16.5,587615000,7.7,"""Mexico City"""
"""South America""",17840000,12.0,428240000,5.6,"""São Paulo"""
"""Australia""",8600000,5.9,41261000,0.5,"""Sydney"""
